In [1]:
import numpy as np
from scipy import stats
from scipy.stats import skew
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.externals import joblib   # 함수는 dump 시켜도 안됨
from sklearn.feature_selection import SelectPercentile, f_classif, f_regression, SelectFromModel, RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [4]:
from my_transformer import change_to_str, divide_columns, feature_selection, simple_imputer, one_hot_encoding, concat, rf_imputer, fill_columns
from my_transformer import rmsle_scorer, neg_rmsle_scorer, rmsle

In [5]:
def preparation(data) :
    
    data.drop(['SalePrice', 'Id'], axis=1, inplace=True)
    
    astype_str = change_to_str('MSSubClass')
    data = astype_str.fit_transform(data)
    
    num_columns, cat_columns = divide_columns(data)
    
    num_null_0 = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                  'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
    num_null_mean = 'LotFrontage'
    num_null_rf = 'GarageYrBlt'
    replace_null = ['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC',
                   'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageFinish', 'GarageCond']
    
    for column in replace_null :
        if column in list(data.columns) :
            data.loc[data[column].isnull(), column] = 'NA'
    
    pipeline_cat = make_pipeline(
        feature_selection(cat_columns),
        simple_imputer('most_frequent'),
        one_hot_encoding(cat_columns)
    )
    
    X_cat = pipeline_cat.fit_transform(data)
    X_num = data[num_columns]
    X = concat(X_num, X_cat)
    
    isnull_sum = X.isnull().sum()
    not_null = list(isnull_sum[isnull_sum == 0].index)
    
    for column in num_columns :
        if column == num_null_rf :
            X = rf_imputer(X, column, not_null)

        elif column == num_null_mean :
            X = fill_columns(X, column, 'mean')

        else :
            X = fill_columns(X, column, 'constant', 0)
        
    isnull_sum = X.isnull().sum()
    print(isnull_sum[isnull_sum > 0].sort_values(ascending=False))
    
    skew_features = X[num_columns].apply(lambda x : skew(x))
    skew_features_top = skew_features[skew_features > 1]
    X[skew_features_top.index] = np.log1p(X[skew_features_top.index])
            
    return X

In [6]:
def data_set() :
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    
    data = pd.concat([train, test], axis=0)
    
    X = preparation(data)
    X_train = X.iloc[:1460, :]
    X_test = X.iloc[1460:, :]
    
    y = train['SalePrice']
    y_train = np.log1p(y)
    
    return X_train, X_test, y_train

In [7]:
X_train, X_test, y_train = data_set()

Series([], dtype: int64)


In [8]:
select = SelectPercentile(score_func=f_regression, percentile=10)
select.fit(X_train, y_train)

SelectPercentile(percentile=10,
         score_func=<function f_regression at 0x00000221B231E730>)

In [9]:
X_train = select.transform(X_train)
X_train.shape

(1460, 32)

In [10]:
X_test = select.transform(X_test)
X_test.shape

(1459, 32)

In [11]:
knn = KNeighborsRegressor(n_neighbors=5)
linear = LinearRegression()
sgd = SGDRegressor(max_iter=None, eta0=0.01, penalty='l2', random_state=30)
ridge = Ridge(alpha=1, random_state=30)
lasso = Lasso(alpha=1, random_state=30)
elastic = ElasticNet(alpha=1, l1_ratio=0.5, random_state=30)
dt = DecisionTreeRegressor(max_depth=None, random_state=30)
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=30, n_jobs=-1)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=30)
svm = SVR(C=1, kernel='rbf', gamma='auto_deprecated')
mlp = MLPRegressor(hidden_layer_sizes=[100], solver='adam', activation='relu', alpha=0.0001, max_iter=200, random_state=30)
xgb = XGBRegressor(random_state=30)

In [12]:
models = {
    'knn' : knn,
    'linear' : linear,
    'sgd' : sgd,
    'ridge' : ridge,
    'lasso' : lasso,
    'elastic' : elastic,
    'dt' : dt,
    'rf' : rf,
    'gb' : gb,
    'xgb' : xgb,
    'svm' : svm,
    'mlp' : mlp
}

In [13]:
scores = dict()

for key, model in models.items() : 
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()   
    scores[key] = round(np.sqrt(-score), 3)
    
scores

[11:03:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'knn': 0.43,
 'linear': 0.398,
 'sgd': 0.671,
 'ridge': 0.397,
 'lasso': 0.4,
 'elastic': 0.4,
 'dt': 0.552,
 'rf': 0.423,
 'gb': 0.405,
 'xgb': 0.402,
 'svm': 0.395,
 'mlp': 0.531}

In [14]:
X_train, X_test, y_train = data_set()

Series([], dtype: int64)


In [15]:
select = SelectFromModel(rf, threshold='1.5*mean')   # mean 에 곱해주는 숫자가 커지면 특성이 줄어든다.
select.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=30, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False,
        threshold='1.5*mean')

In [16]:
X_train = select.transform(X_train)
X_train.shape

(1460, 34)

In [17]:
X_test = select.transform(X_test)
X_test.shape

(1459, 34)

In [18]:
scores = dict()

for key, model in models.items() : 
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()   
    scores[key] = round(np.sqrt(-score), 3)
    
scores

[11:03:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'knn': 0.433,
 'linear': 0.405,
 'sgd': 1.191836881720639e+16,
 'ridge': 0.405,
 'lasso': 0.4,
 'elastic': 0.401,
 'dt': 0.571,
 'rf': 0.407,
 'gb': 0.41,
 'xgb': 0.408,
 'svm': 0.4,
 'mlp': 3.85}

In [19]:
X_train, X_test, y_train = data_set()

Series([], dtype: int64)


In [20]:
select = SelectFromModel(lasso, threshold='0.5*mean')
select.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=30,
   selection='cyclic', tol=0.0001, warm_start=False),
        max_features=None, norm_order=1, prefit=False,
        threshold='0.5*mean')

In [21]:
X_train = select.transform(X_train)
X_train.shape

(1460, 2)

In [22]:
scores = dict()

for key, model in models.items() : 
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()   
    scores[key] = round(np.sqrt(-score), 3)
    
scores

[11:03:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:03:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'knn': 0.436,
 'linear': 0.401,
 'sgd': 572276016058337.5,
 'ridge': 0.401,
 'lasso': 0.4,
 'elastic': 0.4,
 'dt': 0.533,
 'rf': 0.447,
 'gb': 0.405,
 'xgb': 0.404,
 'svm': 0.421,
 'mlp': 1.142}

In [23]:
X_train, X_test, y_train = data_set()

Series([], dtype: int64)


In [24]:
select = RFE(lasso, n_features_to_select=30)
select.fit(X_train, y_train)

RFE(estimator=Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=30,
   selection='cyclic', tol=0.0001, warm_start=False),
  n_features_to_select=30, step=1, verbose=0)

In [25]:
X_train = select.transform(X_train)
X_train.shape

(1460, 30)

In [26]:
scores = dict()

for key, model in models.items() : 
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()   
    scores[key] = round(np.sqrt(-score), 3)
    
scores

[11:04:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:04:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:04:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:04:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:04:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'knn': 0.432,
 'linear': 0.409,
 'sgd': 5984826413272334.0,
 'ridge': 0.407,
 'lasso': 0.4,
 'elastic': 0.401,
 'dt': 0.595,
 'rf': 0.416,
 'gb': 0.413,
 'xgb': 0.41,
 'svm': 0.398,
 'mlp': 4.367}